In [118]:
import pandas as pd
import numpy as np
import csv
from geopy import distance
from math import radians, sin, cos, sqrt, atan2
from heapq import heappop, heappush
from collections import defaultdict

In [119]:
dfDhaka = pd.read_csv("Routemap-BikolpoBus.csv", on_bad_lines='skip')
print(dfDhaka.columns)
dfDhaka.iloc[1]

Index(['DhakaBusBikolpo', '90.364255', '23.828335', '90.364282', '23.827842',
       '90.364282.1', '23.827842.1', '90.364237', '23.827834', '90.364237.1',
       ...
       '90.365139.1', '23.819825.1', '90.365202', '23.819587', '90.365202.1',
       '23.819587.1', '90.365339', '23.819078', 'Mirpur12', 'Purobi Hall'],
      dtype='object', length=123)


DhakaBusBikolpo    DhakaBusBikolpo
90.364255                90.366249
23.828335                23.815764
90.364282                90.366345
23.827842                 23.81541
                        ...       
23.819587.1                    NaN
90.365339                      NaN
23.819078                      NaN
Mirpur12                       NaN
Purobi Hall                    NaN
Name: 1, Length: 123, dtype: object

In [120]:
node_id = 0
edge_id = 0

all_node = []
all_edge = []

def calculate_geo_distance_Vincenty_formula(lat1, lon1, lat2, lon2): 
    return distance.distance((lat1, lon1), (lat2, lon2)).km

def calculate_geo_distance_Haversine_formula2(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = radians(lat1), radians(lon1), radians(lat2), radians(lon2)
    
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    
    radius = 6371
    return radius * c

def calculate_geo_distance_Haversine_formula(u, v):
    return calculate_geo_distance_Haversine_formula2(all_node[u].x, all_node[u].y, all_node[v].x, all_node[v].y)

def calculate_geo_distance_Vincenty_formula(u, v):
    if -90. <= all_node[u].x <= 90.0 and -90. <= all_node[v].x <= 90.0:
        print(u, v)
        return distance.distance((all_node[u].x, all_node[u].y), (all_node[v].x, all_node[v].y)).km
    else:
        return 0

def find_node_id(x, y):
    global node_id
    for i in range(node_id):
        if all_node[i].x == float(x) and all_node[i].y == float(y):
            return i;
    return -1 #not found

class edge:
    def __init__(self, u, v, typ):
        global edge_id
        self.id = edge_id
        self.u = u
        self.v = v
        self.dist = calculate_geo_distance_Haversine_formula(u, v)
        self.type = typ
        edge_id = edge_id + 1
        

class node:
    def __init__(self, x, y):
        global node_id
        self.id = node_id
        self.x = float(x)
        self.y = float(y)
        self.edge = []
        node_id = node_id + 1
        
    def add_edge(self, eg):
        self.edge.append(eg)
        

    
with open('Routemap-UttaraBus.csv', 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for row in csv_reader:
        for i in range(1, len(row) - 2, 2):
            #need a check - is it already a node or not?
            all_node.append(node(row[i], row[i + 1]))
        for i in range(3, len(row) - 2, 2):
            u = find_node_id(row[i - 2], row[i - 1])
            v = find_node_id(row[i], row[i + 1])
            edg = edge(u, v, "UB")
            all_node[u].add_edge(edg)
            all_edge.append(edg)
            #all_node[v].add_edge(edg)  #is it bi-directional?
    
    global node_id, edge_id
    print(node_id, edge_id)

def dijkstra(f, t):
    q, seen = [(0, f, ())], set()
    while q:
        (cost, v1, path) = heappop(q)
        if v1 not in seen:
            seen.add(v1)
            path = (v1, path)
            if v1 == t: return (cost, path)
            
            for edg in all_node[v1].edge:
                c, v2 = edg.dist, edg.v
                if v2 not in seen:
                    heappush(q, (cost + c, v2, path))

    return float("inf")


dijkstra(0, 17)

353 334


(0.0922738538076624, (17, (15, (13, (11, (9, (7, (5, (3, (1, (0, ())))))))))))